In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import multiprocess as mp
root.gStyle.SetOptFit(1111)
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
!rm output/misalignment/?/*
iType = 1
sim = 0
Min_Entries = 100

In [3]:
N_rg = 12
N_files = N_rg
N_arm = 2
N_layer = 8
rebin = 3
if iOption0: rebin = 4
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
#file_path="input/NewHadron/rungroup/"
file_path="input/rg/New/"
#file_name="my-17th_QA_19443_all"
file_name="new_VTX_DC_alignment"
#my-17th_QA_19443_all
#new 7 - only east, 8 - west 2 cm 1.5 gev, 9 - as 8 but 1 Gev; 10 - as 8 but 0.5;  11 -alg corrected;  12 - vtx moved; 13 - vtx and dc moved;  14 - standard
# 19 - sep charge, 20 - 1 iter, 21 - 2 iter, 22 - 3 iter 1.5 pt, 23 - 0.5 pt, 24 - 1.5 phiDC; 25 - 0.5, 26 3 iter 1.5, 27 - 0.5, 28 phi - the corr
hist_select_3D_names = ["dphi_phi0_pt_hist","dthe_the0_pt_hist"]
if iType == 1: hist_select_3D_names = ["dphi_phi0_init_hist","dthe_the0_init_hist","dphi_the0_init_hist","dthe_phi0_init_hist"]
if iType == 2: hist_select_3D_names = ["dphi_phi0_corr_hist","dthe_the0_corr_hist","dphi_the0_corr_hist","dthe_phi0_corr_hist"]
iOptamesX=["phi","the","the","phi"]
iOptamesY=["phi","the","phi","the"]

In [4]:
full_pahtes = ["input/NewHadron/my-m_ee_Run14AuAu_90th_new_19887_300runs.root","../sim/output/Newembed/align/jpsi_v1.root","input/NewHadron/my-m_ee_Run14AuAu_89th_new_19877_398runs.root",\
               "../sim/output/my-kek.root","output/Newembed/deltas/piminus_DCA_50M_V0.root"]
if sim: full_path=full_pahtes[sim-1]

In [5]:
hists_read = []

for ifile in range(N_files):
    infile = root.TFile.Open(file_path+file_name+f"_rg{ifile}.root", "read")
    if sim:
        infile = root.TFile.Open(full_path, "read")
    #print(f"Reading file {ifile} from {file_path+file_name+f'_rg{ifile}.root'}")
    hists_read_file = []
    for ilayer in range(N_layer):
        hist_select_3D0 = []
        NNN = 4
        for ihist in range(NNN):  
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer%4}"))
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"_file{ifile}")
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_read_file.append(hist_select_3D0)
    hists_read.append(hists_read_file)
infile.Close()

In [6]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_gaus = [[-0.051,0.052],[-0.051,0.052]]
boards_pol0 = [[-0.09,0.09],[-0.09,0.09]] 

In [7]:
def misalignment_fitter(InHist, ilayer, iarm, irg):
    N_gaus=1

    hist_phi_bin = InHist.ProjectionY("kek")

    ibinz0 = 1+int(ilayer/4)+iarm*2
    ibinz1 = 1+int(ilayer/4)+iarm*2

    tot_entries = dphi_porj = InHist.ProjectionX(f"dphi_tot_{ilayer}_{iarm}_{irg}", 1, 120, ibinz0, ibinz1).GetEntries()
    N_bins_X = InHist.ProjectionX(f"dphi_tot1_{ilayer}_{iarm}_{irg}", 1, 120, ibinz0, ibinz1).GetNbinsX()

    phi_local =  array( 'd' )
    phi_local_err =  array( 'd' )

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )

    N_bins = hist_phi_bin.GetNbinsX()
    N_iter = int(N_bins/rebin)
 
    for iphi in range(N_iter):
        c1 = root.TCanvas(f"c1_{ilayer}_{iphi}_{iarm}_{irg}",f"c1_{ilayer}_{iphi}_{iarm}_{irg}",720,360)

        ibiny0 = iphi*rebin+1
        ibiny1 = (iphi+1)*rebin

        dphi_porj = InHist.ProjectionX(f"dphi_hist_{ilayer}_{iphi}_{iarm}_{irg}", ibiny0, ibiny1, ibinz0, ibinz1)

        if dphi_porj.GetEntries() < Min_Entries:
            continue

        phi = 0.5* (hist_phi_bin.GetBinCenter(ibiny0) + hist_phi_bin.GetBinCenter(ibiny1))
        phi_err = rebin*hist_phi_bin.GetBinWidth(ibiny0)*0.5

        dphi_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus',  boards_gaus[iarm][0], boards_gaus[iarm][1])
        #g_gaus2= root.TF1( f'g_gaus_2_{ilayer}_{iphi}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol_{ilayer}_{iphi}_{iarm}_{irg}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_pol0[iarm][0], boards_pol0[iarm][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[iarm][0], boards_tot[iarm][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_tot[iarm][1])

        dphi_porj.Draw()
        dphi_porj.Fit( g_gaus, 'QR')
        #dphi_porj.Fit( g_gaus2, 'QR+' )
        dphi_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2] = par_gaus[0], par_gaus[1], par_gaus[2]
        par_tot[3], par_tot[4], par_tot[5] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        #total.SetParLimits(1,boards_tot[iarm][0]/10,boards_tot[iarm][1]/10)
        #total.SetParLimits(2,boards_fit[iarm][1]/10,2)
        
        dphi_porj.Draw()
        dphi_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])

        Format_Func(g_gaus,3,4)

        par_gaus[0], par_gaus[1], par_gaus[2] = par_tot[0], par_tot[1], par_tot[2]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[3], par_tot[4], par_tot[5]
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        if abs(total.GetParameter(1))>0.15 or total.GetNDF()<50 or (total.GetParameter(2)>0.0075 or total.GetParameter(2)<0.002 ): continue

        phi_local.append(phi)
        phi_local_err.append(phi_err)

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1)+total.GetParameter(2)*0.05+abs(total.GetParameter(2))*(dphi_porj.GetEntries()*total.GetNDF()/tot_entries)**(-0.5))
        Sigma_local_err.append(total.GetParError(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs(f"output/misalignment/{iOption0}/fit_{ilayer}_{irg}_{iarm}_{iphi}.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err,phi_local,phi_local_err, int_fg_local, int_bg_local]


In [ ]:
def misalignment_layer_fitter(ifile):
    out_hists = []
    for ilayer in range(N_layer):
        out_hists1 = []
        for iarm in range(N_arm):
            out_hists1.append(misalignment_fitter(hists_read[ifile][ilayer][iOption0], ilayer, iarm, ifile))
        out_hists.append(out_hists1)
    return out_hists

In [9]:
Ntr = N_files

pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(misalignment_layer_fitter,  [i for i in range(Ntr)])
pool.close()

In [10]:
Means, Mean_errs, Sigmas, Sigma_errs, phi_arr, phi_arr_err, Int_FG, Int_BG = [], [], [], [], [], [], [], []
for irg in range(N_rg):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, phi_arr1, phi_arr_err1, Int_FG1, Int_BG1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layer):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, phi_arr0, phi_arr_err0, Int_FG0, Int_BG0 = [], [], [], [], [], [], [], []
        for iarm in range(N_arm):
            Means0.append(output_array[irg][ilayer][iarm][0])
            Mean_errs0.append(output_array[irg][ilayer][iarm][1])
            Sigmas0.append(output_array[irg][ilayer][iarm][2])
            Sigma_errs0.append(output_array[irg][ilayer][iarm][3])
            phi_arr0.append(output_array[irg][ilayer][iarm][4])
            phi_arr_err0.append(output_array[irg][ilayer][iarm][5])
            Int_FG0.append(output_array[irg][ilayer][iarm][6])
            Int_BG0.append(output_array[irg][ilayer][iarm][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        phi_arr1.append(phi_arr0)
        phi_arr_err1.append(phi_arr_err0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    phi_arr.append(phi_arr1)
    phi_arr_err.append(phi_arr_err1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [11]:
root.gStyle.SetOptFit(0000)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []

for irg in range(N_rg):
    fit_functions0 = []
    for ilayer in range(N_layer):
        fit_functions1 = []
        for iarm in range(N_arm):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ilayer}_{irg}_{iarm}","[0]*sin(x)+[1]*cos(x)+[2]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.001)
            fit_functions1[-1].SetParameter(1,0.001)
            fit_functions1[-1].SetParameter(2,0.001)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [13]:
iarm = 0
h2=[]
c2 = root.TCanvas(f"c2_{iarm}",f"c2_{iarm}",1000,400*int(max(N_rg/2,1)))
c2.Divide(2,int(max(N_rg/2,1)))
g_mean = []
min0, max0 = -0.025, 0.045
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    if False and irg ==1 :
        iarm = 1
        if iOption0==0: x0, x1 = -1, 1.5
    g_mean_rg = []
    c2.cd(irg+1)
    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}0 (rad)", f"#LT d{iOptamesY[iOption0]} #GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h2[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_layer,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Means[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions[irg][ilayer][iarm],"Q","")
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean_layer,f"neg layer {ilayer}","p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c2.Draw()
c2.SaveAs(f"output/misalignment/note/mean_{iType}_{iOption0}.pdf")

In [14]:
iarm = 1
h2=[]
c3 = root.TCanvas(f"c3_{iarm}",f"c3_{iarm}",1000,400*int(max(N_rg/2,1)))
c3.Divide(2,int(max(N_rg/2,1)))
g_mean = []
min0, max0 = -0.015, 0.015
x0, x1 = -1, 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    g_mean_rg = []
    c3.cd(irg+1)
    h2.append(Format_Pad_old( x0, x1, min0+0.008, max0, f"{iOptamesX[iOption0]}0 (rad)", f"#LT d{iOptamesY[iOption0]} #GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h2[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_layer,1):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Means[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Mean_errs[irg][ilayer][iarm])
        Format_Graph(g_mean_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean_layer.Fit(fit_functions[irg][ilayer][iarm],"Q","")
        g_mean_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean_layer,f"neg layer {ilayer}","p")
        g_mean_rg.append(g_mean_layer)
    legends[-1].Draw()
    g_mean.append(g_mean_rg)
c3.Draw()
c3.SaveAs(f"output/misalignment/mean_{iarm}_{iOption0}.png")

In [15]:
if True:
    ilayer = 2
    print(f"const float "+iOptamesX[iOption0]+f"_offset_params[{N_rg}][{N_arm}][2][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            iirg = irg
            #if iOption0 == 1 and irg>4 and iarm == 0:
            #    if irg < 8: iirg = 4
            #    elif irg < 10: iirg = 11
            print("        {")     
            print("            {"+f"{round(fit_functions[iirg][ilayer+0][iarm].GetParameter(0),6)}, {round(fit_functions[iirg][ilayer+0][iarm].GetParameter(1),6)}, {round(fit_functions[iirg][ilayer+0][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[iirg][ilayer+4][iarm].GetParameter(0),6)}, {round(fit_functions[iirg][ilayer+4][iarm].GetParameter(1),6)}, {round(fit_functions[iirg][ilayer+4][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")



const float phi_offset_params[12][2][2][3] = {
    {
        {
            {-0.005687, -0.00268, -0.005751},
            {-0.006909, 0.000672, -0.005335},
        },
        {
            {0.001562, -0.001542, 0.009316},
            {0.002183, -0.003283, 0.007963},
        },
    },
    {
        {
            {-0.005953, -0.002369, -0.005552},
            {-0.00719, 0.000734, -0.005388},
        },
        {
            {0.002003, -0.000704, 0.008644},
            {0.002701, -0.003328, 0.007902},
        },
    },
    {
        {
            {-0.005311, -0.002446, -0.00575},
            {-0.006722, 0.001541, -0.004662},
        },
        {
            {0.002351, -0.000381, 0.008314},
            {0.002918, -0.002573, 0.007299},
        },
    },
    {
        {
            {-0.005662, -0.002477, -0.005785},
            {-0.006983, 0.000944, -0.005239},
        },
        {
            {0.002231, -0.00041, 0.008338},
            {0.002437, -0.003646, 0.008258},
        },
    },
    {

In [16]:
if False:
    print(f"const float "+iOptames[iOption0]+f"_offset_param0[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(0),6)}, {round(fit_functions[1][irg][iarm].GetParameter(0),6)}, {round(fit_functions[2][irg][iarm].GetParameter(0),6)}, {round(fit_functions[3][irg][iarm].GetParameter(0),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(0),6)}, {round(fit_functions[5][irg][iarm].GetParameter(0),6)}, {round(fit_functions[6][irg][iarm].GetParameter(0),6)}, {round(fit_functions[7][irg][iarm].GetParameter(0),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param1[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(1),6)}, {round(fit_functions[1][irg][iarm].GetParameter(1),6)}, {round(fit_functions[2][irg][iarm].GetParameter(1),6)}, {round(fit_functions[3][irg][iarm].GetParameter(1),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(1),6)}, {round(fit_functions[5][irg][iarm].GetParameter(1),6)}, {round(fit_functions[6][irg][iarm].GetParameter(1),6)}, {round(fit_functions[7][irg][iarm].GetParameter(1),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param2[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(2),6)}, {round(fit_functions[1][irg][iarm].GetParameter(2),6)}, {round(fit_functions[2][irg][iarm].GetParameter(2),6)}, {round(fit_functions[3][irg][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(2),6)}, {round(fit_functions[5][irg][iarm].GetParameter(2),6)}, {round(fit_functions[6][irg][iarm].GetParameter(2),6)}, {round(fit_functions[7][irg][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")     
     


In [17]:
if False:
    ilayer, ilayer1 = 1, 5
    if iOption0: print(f"const float the_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    else: print(f"const float phi_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {"+f"{round(fit_functions[ilayer][irg][iarm].GetParameter(0)+fit_functions[ilayer1][irg][iarm].GetParameter(0),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(1)+fit_functions[ilayer1][irg][iarm].GetParameter(1),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(2)+fit_functions[ilayer1][irg][iarm].GetParameter(2),6)}"+"},")
        print("    },")   
    print("};")        

In [18]:
iarm = 0
h4=[]
c4 = root.TCanvas(f"c4_{iarm}",f"c4_{iarm}",1000,400*int(N_rg/2))
c4.Divide(2,int(N_rg/2))
min0, max0 = -0.015, 0.015
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
g_sigma = []
for irg in range(N_rg):
    c4.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max0, f"{iOptamesX[iOption0]}0 (rad)", f"sigma d{iOptamesY[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    g_sigma_rg = []
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_sigma_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Sigmas[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Sigma_errs[irg][ilayer][iarm])
        Format_Graph(g_sigma_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_sigma_layer.Fit("pol0","Q","")
        g_sigma_layer.GetFunction("pol0").SetLineColor(colors[ilayer])
        g_sigma_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_sigma_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_sigma_layer,f"neg layer {ilayer}","p")
        g_sigma_rg.append(g_sigma_layer)
    g_sigma.append(g_sigma_rg)
    legends[-1].Draw()
c4.Draw()
c4.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")

In [19]:
iarm = 1
h4=[]
c5 = root.TCanvas(f"c5_{iarm}",f"c5_{iarm}",1000,400*int(N_rg/2))
c5.Divide(2,int(N_rg/2))
min0, max0 = -0.015, 0.015
x0, x1 =-1, 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
g_sigma = []
for irg in range(N_rg):
    c5.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max0, f"{iOptamesX[iOption0]}0 (rad)", f"sigma d{iOptamesY[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    g_sigma_rg = []
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[irg][ilayer][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_sigma_layer = root.TGraphErrors(N_phi, phi_arr[irg][ilayer][iarm], Sigmas[irg][ilayer][iarm],phi_arr_err[irg][ilayer][iarm],Sigma_errs[irg][ilayer][iarm])
        Format_Graph(g_sigma_layer,20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_sigma_layer.Fit("pol0","Q","")
        g_sigma_layer.GetFunction("pol0").SetLineColor(colors[ilayer])
        g_sigma_layer.Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_sigma_layer,f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_sigma_layer,f"neg layer {ilayer}","p")
        g_sigma_rg.append(g_sigma_layer)
    g_sigma.append(g_sigma_rg)
    legends[-1].Draw()
c5.Draw()
c5.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")